In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import pandas as pd
import numpy as np

In [2]:
table_tmp = np.genfromtxt('../calibrators/Hriz_3sigClip.csv' , delimiter=',', 
                        filling_values=-1, names=True, dtype=None, encoding=None)
pgc_drop_riz = table_tmp['PGC']

table_tmp = np.genfromtxt('../calibrators/Hw1_3sigClip.csv' , delimiter=',', 
                        filling_values=-1, names=True, dtype=None, encoding=None)
pgc_drop_w1 = table_tmp['PGC']

In [3]:
drop = np.genfromtxt('../calibrators/TF_drop.csv' , delimiter='|', 
                        filling_values=-1, names=True, dtype=None, encoding=None)
pgc_drop = drop['PGC']

faint = np.genfromtxt('../calibrators/TF_faint.csv' , delimiter='|', 
                        filling_values=-1, names=True, dtype=None, encoding=None)
pgc_faint = faint['PGC']

pgc_reject = np.concatenate((pgc_drop, pgc_faint))
pgc_reject = np.unique(pgc_reject)

In [4]:
df = pd.read_csv('2mtf.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_2mtf = df.set_index('PGC')

df_2mtf.head()

,Name,Dadj,DMadj,eDM,2mass,RA,Dec,cz2m,Ktc,Htc,...,It,Kt,V,Ag,Nest,PGC1,Ng,Vgm,Sigp,R2t
PGC,,,,,,,,,,,,,,,,,,,,,
70,UGC12900,120.4,35.402,0.388,00005604+2020165,0.23345,20.33799,6804,11.124,11.616,...,13.88,11.19,6804,0.34,209949,70,1,6561,122,0.249
76,UGC12901,106.9,35.146,0.461,00005891+2854421,0.24550,28.91172,6899,10.818,11.085,...,12.80,10.89,6896,0.21,209247,76,1,6676,139,0.282
80,IC5375,134.8,35.648,0.455,00010478+0432261,0.26987,4.54060,9151,11.029,11.274,...,0.00,11.06,9223,0.13,207489,80,1,8992,182,0.371
102,IC5376,84.0,34.621,0.414,00011976+3431326,0.33238,34.52571,5032,10.405,10.704,...,0.00,10.47,5029,0.31,210310,102,1,4769,114,0.232
120,UGC12914,98.3,34.963,0.377,00013830+2329011,0.40968,23.48363,4371,9.330,9.591,...,11.50,9.39,4349,0.48,202766,120,2,4048,170,0.345


In [5]:
df = pd.read_csv('spitzer_cf3_tfr.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_spitzer = df.set_index('PGC')

df_spitzer.head()

,pgc1,Nest,Name,Glon,Glat,SGL,SGB,T,b/a,Inc,...,DMcc,DMccb,Mcal,DMnc,DMncb,Dcf3,eD,HP,logH,C
PGC,,,,,,,,,,,,,,,,,,,,,
94,94,0,UGC12905,120.8368,17.9719,16.8900,17.6383,6.1,0.12,90,...,NaN,NaN,-18.155,34.098,34.152,67.7,18.3,64.5,1.810,0
181,181,0,ESO012-014,305.4559,-36.5399,216.1117,-13.3206,9.0,0.40,69,...,NaN,NaN,-15.555,30.415,30.415,12.1,3.3,144.1,2.159,0
218,218,211714,NGC7814,106.4095,-45.1746,309.0616,16.4022,2.0,0.20,90,...,31.227,31.227,-21.886,31.444,31.444,17.6,4.0,72.5,1.860,1
255,218,211714,UGC00017,106.2151,-46.0989,308.1423,16.0957,9.1,0.61,54,...,31.303,31.303,-16.301,31.083,31.083,18.2,4.2,60.4,1.781,1
279,279,202396,NGC7817,108.2283,-40.7605,313.8141,17.1420,4.1,0.27,79,...,32.352,32.360,-21.530,32.379,32.387,29.7,6.8,85.9,1.934,1


In [6]:
df = pd.read_csv('all_cf2_tfr.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_allcf2 = df.set_index('PGC')

df_allcf2.head()

,Dist,Name,RAJ,DeJ,Glon,Glat,SGL,SGB,Ty,Asfd,...,DM,Nest,PGC1,Ndg,DMg,eDMg,NVg,Vhg,Vlsg,V3kg
PGC,,,,,,,,,,,,,,,,,,,,,
4,50.6,AGC331060,3.5,230515.5,107.8322,-38.2729,316.0587,18.4514,5,0.40,...,33.52,202766,120,1,33.52,0.4,2,4353,4601,4005
55,73.8,UGC12898,37.4,333603.4,110.9496,-28.0856,327.0998,19.7763,6,0.22,...,34.34,0,55,1,34.34,0.4,1,4779,5052,4454
70,117.5,UGC12900,56.1,202016.7,107.1780,-40.9837,313.2487,17.7662,6,0.34,...,35.35,209949,70,1,35.35,0.4,1,6800,7040,6447
76,97.7,UGC12901,58.9,285441.5,109.8059,-32.6707,322.1728,19.1316,3,0.21,...,34.95,209247,76,1,34.95,0.4,1,6920,7183,6583
124,81.7,UGC12913,136.7,33020.1,99.5866,-57.0904,296.2413,13.7943,6,0.10,...,34.56,0,124,1,34.56,0.4,1,6350,6529,5988


In [7]:
df = pd.read_csv('6dfgs.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_6dfgs = df.set_index('PGC')

df_6dfgs.head()

,PGC1,Nest,Name,Vcmb,Vgp,D75,D100,feD,DzDh,eDz,skew,RA,DE,Vg,Gp
PGC,,,,,,,,,,,,,,,
27,27,0,g0000235-065610,10920,10920,145.5,109.2,0.27,0.015698,0.105029,-0.021398,0.0978,-6.9362,10920,-1
40,40,209793,g0000356-014547,6956,6956,116.4,87.3,0.31,-0.088660,0.118089,0.016330,0.1485,-1.7632,6956,-1
51,142,201987,g0000358-403432,14746,14746,248.6,186.5,0.42,-0.081111,0.152326,0.015862,0.1489,-40.5756,14746,-1
64,72642,200033,g0000523-355037,15324,14646,222.3,166.7,0.29,-0.034659,0.110224,0.010894,0.2181,-35.8437,14646,1261
66,72642,200033,g0000532-355911,14725,14646,191.0,143.2,0.27,0.030506,0.103811,-0.028422,0.2216,-35.9863,14646,1261


In [8]:
df = pd.read_csv('flat_tfr.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_flat = df.set_index('PGC')

df_flat.head()

,DMadj,DMbig,DMfull,Vh,Vls,Vcmb,Vmod,f,PGC1,Nest,glon,glat,sgl,sgb,DMg,eDMg
PGC,,,,,,,,,,,,,,,,
70,35.384,34.817,34.859,6800,7040,6447,6557,1.017,70,209949,107.1780,-40.9837,313.2487,17.7662,35.12,0.12
642,33.097,32.780,32.572,4441,4718,4130,4175,1.011,642,210102,113.6137,-24.6495,331.3316,18.5421,34.06,0.19
731,35.444,34.926,34.919,7059,7318,6725,6845,1.018,791,200264,112.2328,-33.0307,322.5668,17.0954,35.23,0.13
978,33.699,33.290,33.174,5399,5626,5049,5116,1.013,978,207997,110.8971,-43.4288,312.1216,14.2463,34.33,0.16
1422,34.619,34.141,34.094,5663,5893,5319,5393,1.014,1351,200045,113.7645,-41.7438,314.5450,12.9247,34.59,0.14


In [9]:
df = pd.read_csv('../calibrators/DMs_EDD.csv.test', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df = df.rename(columns=lambda x: x if x!='DM_w1' else 'DMw1')
df = df.rename(columns=lambda x: x if x!='dDM_w1' else 'dDMw1')
df = df.rename(columns=lambda x: x if x!='pgc' else 'PGC')
df = df.set_index('PGC')

## Using hugh quality distances, photoemtry, etc to determine zero points
df_cf4tfr = df[((((df['Sqlt']>2) & ~df.index.isin(pgc_drop_riz)) | 
          ((df['Wqlt']>2) & ~df.index.isin(pgc_drop_w1))) & 
          (~df.index.isin(pgc_reject)))]

print len(df_cf4tfr)
df_cf4tfr.head()

9693


,Vh,Vls,V3k,Vmod,f_cosmology,logWimx,logWimx_e,inc,Sqlt,Wqlt,...,pgc1,Nest,Vmodg,Nkt,N15,sglg,sgbg,RS,P0,DM_src
PGC,,,,,,,,,,,,,,,,,,,,,
2,5004,5296,4726,4786,1.013,2.744,0.029,52,0,5,...,73150,200275,4976,0,7,341.4922,20.7395,2,2.02,2
4,4458,4706,4109,4154,1.011,2.189,0.014,85,5,4,...,120,202766,4154,0,0,316.0587,18.4514,2,-2.75,1
12,6548,6685,6195,6297,1.016,2.606,0.021,82,0,5,...,12,210177,6279,0,1,286.4249,11.3510,2,0.56,2
16,5667,5809,5312,5387,1.014,2.515,0.025,65,5,5,...,16,211419,5434,0,1,287.6120,11.7030,2,0.92,1
55,4779,5052,4454,4507,1.012,2.260,0.025,80,5,4,...,55,0,4507,0,0,327.0996,19.7763,2,-2.83,1


In [10]:
df = pd.read_csv('6dfgs.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_6dfgs = df.set_index('PGC')

ln10 = np.log(10.)
df_6dfgs['DM_6dfgs']  = (5*np.log10(df_6dfgs.D75)+25.)
df_6dfgs['eDM_6dfgs'] = 5.*df_6dfgs.feD/ln10

# df_6dfgs = df_6dfgs.rename(columns={"Nest": "Nest_6dfgs"})
df_6dfgs.head()

,PGC1,Nest,Name,Vcmb,Vgp,D75,D100,feD,DzDh,eDz,skew,RA,DE,Vg,Gp,DM_6dfgs,eDM_6dfgs
PGC,,,,,,,,,,,,,,,,,
27,27,0,g0000235-065610,10920,10920,145.5,109.2,0.27,0.015698,0.105029,-0.021398,0.0978,-6.9362,10920,-1,35.814315,0.586298
40,40,209793,g0000356-014547,6956,6956,116.4,87.3,0.31,-0.088660,0.118089,0.016330,0.1485,-1.7632,6956,-1,35.329765,0.673156
51,142,201987,g0000358-403432,14746,14746,248.6,186.5,0.42,-0.081111,0.152326,0.015862,0.1489,-40.5756,14746,-1,36.977506,0.912018
64,72642,200033,g0000523-355037,15324,14646,222.3,166.7,0.29,-0.034659,0.110224,0.010894,0.2181,-35.8437,14646,1261,36.734697,0.629727
66,72642,200033,g0000532-355911,14725,14646,191.0,143.2,0.27,0.030506,0.103811,-0.028422,0.2216,-35.9863,14646,1261,36.405167,0.586298


In [11]:
# print(df_cf4tfr.columns)
# print(len(df_cf4tfr))

In [12]:
# 37719 in pgc_drop_riz

In [13]:
# df_cf4tfr.loc[37719]

In [14]:
# print len(df)
# df.iloc[0:10]

In [15]:
print(df_allcf2.columns)

Index([u'Dist', u'Name', u'RAJ', u'DeJ', u'Glon', u'Glat', u'SGL', u'SGB',
       u'Ty', u'Asfd', u'Asf', u'Btot', u'Itot', u'Ks', u'b/a', u'Vhel',
       u'Vgsr', u'Vls', u'Vcmb', u'Icnt', u'2M++', u'Fil', u'Ibik', u'Inc',
       u'Wmxi', u'logW', u'DMcf2', u'Dcf2', u'm_c', u'In', u'lgWtf', u'DMsfi',
       u'Dsfi', u'DM', u'Nest', u'PGC1', u'Ndg', u'DMg', u'eDMg', u'NVg',
       u'Vhg', u'Vlsg', u'V3kg'],
      dtype='object')


# All CF2: Distance modulus: average of CF2 (given double weight) and SFI++; mid_IR calibration

In [16]:
df = df_allcf2[["Dist", "DM", "DMcf2", "DMsfi"]]
df["Mmean"] = (2.*df_allcf2.DMcf2 + df_allcf2.DMsfi)/3.

df["Dmean"] = 10**((df["Mmean"]-25)/5)

df.head()

/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Dist,DM,DMcf2,DMsfi,Mmean,Dmean
PGC,,,,,,
4,50.6,33.52,33.46,33.64,33.520000,50.582466
55,73.8,34.34,34.25,34.52,34.340000,73.790423
70,117.5,35.35,35.32,35.40,35.346667,117.309540
76,97.7,34.95,35.03,34.80,34.953333,97.873849
124,81.7,34.56,34.55,34.58,34.560000,81.658237


In [17]:
# df_2mtf
# df_spitzer
# df_allcf2
# df_cf4tfr

In [18]:
print(df_cf4tfr.columns)
print(len(df_cf4tfr))

Index([u'Vh', u'Vls', u'V3k', u'Vmod', u'f_cosmology', u'logWimx',
       u'logWimx_e', u'inc', u'Sqlt', u'Wqlt', u'DM_best', u'dDM_best', u'DMr',
       u'dDMr', u'RMS_r', u'DMr_raw', u'DMi', u'dDMi', u'RMS_i', u'DMi_raw',
       u'DMz', u'dDMz', u'RMS_z', u'DMz_raw', u'DMw1', u'dDMw1', u'RMS_w1',
       u'DMw1_raw', u'ggg', u'rrr', u'iii', u'zzz', u'w1', u'w2', u'RA',
       u'DEC', u'gl', u'gb', u'sgl', u'sgb', u'pgc1', u'Nest', u'Vmodg',
       u'Nkt', u'N15', u'sglg', u'sgbg', u'RS', u'P0', u'DM_src'],
      dtype='object')
9693


In [19]:
df = df_cf4tfr.join(df_flat, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_flat')

print(df.columns)
print(len(df))


df = df[(df.Nest_cf4 != df.Nest_flat) | (df.pgc1 != df.PGC1)]

df[["Nest_cf4", "pgc1", "Nest_flat", "PGC1"]]

Index([u'Vh_cf4', u'Vls_cf4', u'V3k', u'Vmod_cf4', u'f_cosmology', u'logWimx',
       u'logWimx_e', u'inc', u'Sqlt', u'Wqlt', u'DM_best', u'dDM_best', u'DMr',
       u'dDMr', u'RMS_r', u'DMr_raw', u'DMi', u'dDMi', u'RMS_i', u'DMi_raw',
       u'DMz', u'dDMz', u'RMS_z', u'DMz_raw', u'DMw1', u'dDMw1', u'RMS_w1',
       u'DMw1_raw', u'ggg', u'rrr', u'iii', u'zzz', u'w1', u'w2', u'RA',
       u'DEC', u'gl', u'gb', u'sgl_cf4', u'sgb_cf4', u'pgc1', u'Nest_cf4',
       u'Vmodg', u'Nkt', u'N15', u'sglg', u'sgbg', u'RS', u'P0', u'DM_src',
       u'DMadj', u'DMbig', u'DMfull', u'Vh_flat', u'Vls_flat', u'Vcmb',
       u'Vmod_flat', u'f', u'PGC1', u'Nest_flat', u'glon', u'glat',
       u'sgl_flat', u'sgb_flat', u'DMg', u'eDMg'],
      dtype='object')
491


,Nest_cf4,pgc1,Nest_flat,PGC1
PGC,,,,


In [20]:
df = df_cf4tfr.join(df_6dfgs, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_6dfgs')

print(df.columns)
print(len(df))


df = df[(df.Nest_cf4 != df.Nest_6dfgs) | (df.pgc1 != df.PGC1)]

df[["Nest_cf4", "pgc1", "Nest_6dfgs", "PGC1"]]

Index([u'Vh', u'Vls', u'V3k', u'Vmod', u'f_cosmology', u'logWimx',
       u'logWimx_e', u'inc', u'Sqlt', u'Wqlt', u'DM_best', u'dDM_best', u'DMr',
       u'dDMr', u'RMS_r', u'DMr_raw', u'DMi', u'dDMi', u'RMS_i', u'DMi_raw',
       u'DMz', u'dDMz', u'RMS_z', u'DMz_raw', u'DMw1', u'dDMw1', u'RMS_w1',
       u'DMw1_raw', u'ggg', u'rrr', u'iii', u'zzz', u'w1', u'w2', u'RA_cf4',
       u'DEC', u'gl', u'gb', u'sgl', u'sgb', u'pgc1', u'Nest_cf4', u'Vmodg',
       u'Nkt', u'N15', u'sglg', u'sgbg', u'RS', u'P0', u'DM_src', u'PGC1',
       u'Nest_6dfgs', u'Name', u'Vcmb', u'Vgp', u'D75', u'D100', u'feD',
       u'DzDh', u'eDz', u'skew', u'RA_6dfgs', u'DE', u'Vg', u'Gp', u'DM_6dfgs',
       u'eDM_6dfgs'],
      dtype='object')
71


,Nest_cf4,pgc1,Nest_6dfgs,PGC1
PGC,,,,


In [21]:
df = df_cf4tfr.join(df_allcf2, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_cf2')

print(df.columns)
print(len(df))

# df[["Nest_cf4", "pgc1", "Nest_cf2", "PGC1"]].head(10)

df = df[(df.Nest_cf4 != df.Nest_cf2) | (df.pgc1 != df.PGC1)]

df[["Nest_cf4", "pgc1", "Nest_cf2", "PGC1"]]

Index([u'Vh', u'Vls_cf4', u'V3k', u'Vmod', u'f_cosmology', u'logWimx',
       u'logWimx_e', u'inc', u'Sqlt', u'Wqlt', u'DM_best', u'dDM_best', u'DMr',
       u'dDMr', u'RMS_r', u'DMr_raw', u'DMi', u'dDMi', u'RMS_i', u'DMi_raw',
       u'DMz', u'dDMz', u'RMS_z', u'DMz_raw', u'DMw1', u'dDMw1', u'RMS_w1',
       u'DMw1_raw', u'ggg', u'rrr', u'iii', u'zzz', u'w1', u'w2', u'RA',
       u'DEC', u'gl', u'gb', u'sgl', u'sgb', u'pgc1', u'Nest_cf4', u'Vmodg',
       u'Nkt', u'N15', u'sglg', u'sgbg', u'RS', u'P0', u'DM_src', u'Dist',
       u'Name', u'RAJ', u'DeJ', u'Glon', u'Glat', u'SGL', u'SGB', u'Ty',
       u'Asfd', u'Asf', u'Btot', u'Itot', u'Ks', u'b/a', u'Vhel', u'Vgsr',
       u'Vls_cf2', u'Vcmb', u'Icnt', u'2M++', u'Fil', u'Ibik', u'Inc', u'Wmxi',
       u'logW', u'DMcf2', u'Dcf2', u'm_c', u'In', u'lgWtf', u'DMsfi', u'Dsfi',
       u'DM', u'Nest_cf2', u'PGC1', u'Ndg', u'DMg', u'eDMg', u'NVg', u'Vhg',
       u'Vlsg', u'V3kg'],
      dtype='object')
3701


,Nest_cf4,pgc1,Nest_cf2,PGC1
PGC,,,,


In [22]:
df = df_cf4tfr.join(df_spitzer, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_spitzer')
# print(df.columns)
# print(len(df))

# df[["pgc1_cf4", "Nest_cf4", "pgc1_spitzer", "Nest_spitzer"]].head(20)

df = df[(df.Nest_cf4 != df.Nest_spitzer) | (df.pgc1_cf4 != df.pgc1_spitzer)]

df = df[["Nest_cf4", "pgc1_cf4", "Nest_spitzer", "pgc1_spitzer"]]

df

,Nest_cf4,pgc1_cf4,Nest_spitzer,pgc1_spitzer
PGC,,,,


In [23]:
df = df_cf4tfr.join(df_2mtf, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_2mtf')
print(df.columns)
print(len(df))

# df[["pgc1", "Nest_cf4", "PGC1", "Nest_2mtf"]].head(10)

df = df[(df.Nest_cf4 != df.Nest_2mtf) | (df.pgc1 != df.PGC1)]

df[["Nest_cf4", "pgc1", "Nest_2mtf", "PGC1"]]

Index([u'Vh', u'Vls', u'V3k', u'Vmod', u'f_cosmology', u'logWimx',
       u'logWimx_e', u'inc', u'Sqlt', u'Wqlt',
       ...
       u'It', u'Kt', u'V', u'Ag', u'Nest_2mtf', u'PGC1', u'Ng', u'Vgm',
       u'Sigp', u'R2t'],
      dtype='object', length=104)
1495


,Nest_cf4,pgc1,Nest_2mtf,PGC1
PGC,,,,


In [24]:
df = df_cf4tfr.join(df_allcf2, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_cf2')

print(df.columns)
print(len(df))

Ye = 0.4 + 0. * df.DMsfi
YE = df.dDM_best
err = np.sqrt(Ye**2+YE**2)
We = 1./err**2

Ysfi = df.DM_best - df.DMsfi
Ycf2 = df.DM_best - df.DMcf2
Y    = df.DM_best - df.DM

print "sfi: ", np.median(Ysfi[df.DMsfi>0]), np.average(Ysfi[df.DMsfi>0], weights=We[df.DMsfi>0])
print "cf2: ", np.median(Ycf2[df.DMcf2>0]), np.average(Ycf2[df.DMcf2>0], weights=We[df.DMcf2>0])
print "best: ", np.median(Y[df.DM>0]), np.average(Y[df.DM>0], weights=We[df.DM>0])

Index([u'Vh', u'Vls_cf4', u'V3k', u'Vmod', u'f_cosmology', u'logWimx',
       u'logWimx_e', u'inc', u'Sqlt', u'Wqlt', u'DM_best', u'dDM_best', u'DMr',
       u'dDMr', u'RMS_r', u'DMr_raw', u'DMi', u'dDMi', u'RMS_i', u'DMi_raw',
       u'DMz', u'dDMz', u'RMS_z', u'DMz_raw', u'DMw1', u'dDMw1', u'RMS_w1',
       u'DMw1_raw', u'ggg', u'rrr', u'iii', u'zzz', u'w1', u'w2', u'RA',
       u'DEC', u'gl', u'gb', u'sgl', u'sgb', u'pgc1', u'Nest_cf4', u'Vmodg',
       u'Nkt', u'N15', u'sglg', u'sgbg', u'RS', u'P0', u'DM_src', u'Dist',
       u'Name', u'RAJ', u'DeJ', u'Glon', u'Glat', u'SGL', u'SGB', u'Ty',
       u'Asfd', u'Asf', u'Btot', u'Itot', u'Ks', u'b/a', u'Vhel', u'Vgsr',
       u'Vls_cf2', u'Vcmb', u'Icnt', u'2M++', u'Fil', u'Ibik', u'Inc', u'Wmxi',
       u'logW', u'DMcf2', u'Dcf2', u'm_c', u'In', u'lgWtf', u'DMsfi', u'Dsfi',
       u'DM', u'Nest_cf2', u'PGC1', u'Ndg', u'DMg', u'eDMg', u'NVg', u'Vhg',
       u'Vlsg', u'V3kg'],
      dtype='object')
3701
sfi:  -0.11000000000000476 -0.1894

In [25]:
df['DMw'] = df.DM_best/df.dDM_best**2
df['DMwn'] = 1./df.dDM_best**2

gf = df.groupby(['pgc1']).agg({'DMw':'sum', 'DMwn':'sum', 'DM_best': ['mean','count'], 'DM': 'mean'})
gf['DMwav'] = gf.DMw / gf.DMwn

gf['Err_DM'] = 0.4/np.sqrt(gf.DM_best.iloc[:,1])
gf['Err_DMwav'] = 1. / np.sqrt(gf.DMwn)

## Groups with more than 3 galaxies
gf = gf[gf.DM_best.iloc[:,1]>3]

Y   = gf.DMwav - gf.DM.iloc[:,0]
err = np.sqrt(gf.Err_DM**2+gf.Err_DMwav**2)
We = 1./err**2
print "best: ", np.median(Y), np.average(Y, weights=We)



best:  -0.13970290843890254 -1.303856547358919


In [26]:
# df.agg()?
pd.NamedAgg?

Object `pd.NamedAgg` not found.


In [27]:
df.groupby(['pgc1']).agg

<bound method DataFrameGroupBy.aggregate of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7f3168cc3710>>

In [28]:
# df_2mtf
# df_spitzer
# df_allcf2
# df_cf4tfr

In [29]:
df = df_cf4tfr.join(df_6dfgs, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_2mtf')
print(df.columns)
print(len(df))

# # df[["pgc1", "Nest_cf4", "PGC1", "Nest_2mtf"]].head(10)

df = df[(df.Nest_cf4 != df.Nest_2mtf) | (df.pgc1 != df.PGC1)]

df[["Nest_cf4", "pgc1", "Nest_2mtf", "PGC1"]]

Index([u'Vh', u'Vls', u'V3k', u'Vmod', u'f_cosmology', u'logWimx',
       u'logWimx_e', u'inc', u'Sqlt', u'Wqlt', u'DM_best', u'dDM_best', u'DMr',
       u'dDMr', u'RMS_r', u'DMr_raw', u'DMi', u'dDMi', u'RMS_i', u'DMi_raw',
       u'DMz', u'dDMz', u'RMS_z', u'DMz_raw', u'DMw1', u'dDMw1', u'RMS_w1',
       u'DMw1_raw', u'ggg', u'rrr', u'iii', u'zzz', u'w1', u'w2', u'RA_cf4',
       u'DEC', u'gl', u'gb', u'sgl', u'sgb', u'pgc1', u'Nest_cf4', u'Vmodg',
       u'Nkt', u'N15', u'sglg', u'sgbg', u'RS', u'P0', u'DM_src', u'PGC1',
       u'Nest_2mtf', u'Name', u'Vcmb', u'Vgp', u'D75', u'D100', u'feD',
       u'DzDh', u'eDz', u'skew', u'RA_2mtf', u'DE', u'Vg', u'Gp', u'DM_6dfgs',
       u'eDM_6dfgs'],
      dtype='object')
71


,Nest_cf4,pgc1,Nest_2mtf,PGC1
PGC,,,,


In [30]:
df = df_cf4tfr.join(df_spitzer, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_spitzer')

print(df.columns)
print(len(df))

ln10 = np.log(10.)

Ye = 5*df.eD/df.Dcf3/ln10
YE = df.dDM_best
err = np.sqrt(Ye**2+YE**2)
We = 1./err**2
Y = df.DM_best - df.DMccb
print "best: ", np.median(Y[(df.DMccb>0 &(df.Wqlt>2))]), np.average(Y[(df.DMccb>0 &(df.Wqlt>2))], 
                                                                    weights=We[(df.DMccb>0 &(df.Wqlt>2))])




Ye = 5*df.eD/df.Dcf3/ln10
YE = df.dDM_best
err = np.sqrt(Ye**2+YE**2)
We = 1./err**2
Y = df.DM_best - (5*np.log10(df.Dcf3)+25)
print "best: ", np.median(Y[df.Dcf3>0]), np.average(Y[df.Dcf3>0], weights=We[df.Dcf3>0])

Index([u'Vh', u'Vls_cf4', u'V3k', u'Vmod_cf4', u'f_cosmology', u'logWimx',
       u'logWimx_e', u'inc', u'Sqlt', u'Wqlt', u'DM_best', u'dDM_best', u'DMr',
       u'dDMr', u'RMS_r', u'DMr_raw', u'DMi', u'dDMi', u'RMS_i', u'DMi_raw',
       u'DMz', u'dDMz', u'RMS_z', u'DMz_raw', u'DMw1', u'dDMw1', u'RMS_w1',
       u'DMw1_raw', u'ggg', u'rrr', u'iii', u'zzz', u'w1', u'w2', u'RA',
       u'DEC', u'gl', u'gb', u'sgl', u'sgb', u'pgc1_cf4', u'Nest_cf4',
       u'Vmodg', u'Nkt', u'N15', u'sglg', u'sgbg', u'RS', u'P0', u'DM_src',
       u'pgc1_spitzer', u'Nest_spitzer', u'Name', u'Glon', u'Glat', u'SGL',
       u'SGB', u'T', u'b/a', u'Inc', u'Ag', u'Itot', u'Ibik', u'[3.6]',
       u'[3.6]c', u'CC', u'[3.6]cc', u'Vhel', u'Vls_spitzer', u'Vcmb',
       u'Vmod_spitzer', u'Wmx', u'eW', u'Wmxi', u'logW', u'elogW', u'Ccal',
       u'DMcc', u'DMccb', u'Mcal', u'DMnc', u'DMncb', u'Dcf3', u'eD', u'HP',
       u'logH', u'C'],
      dtype='object')
1967
best:  -0.08099999999999774 -0.16513751763717055
b

In [31]:
df = pd.read_csv('all_cf2_fp.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_cf2fp = df.set_index('PGC')

df = df_cf4tfr.join(df_cf2fp, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_cf2fp')

print(df.columns)
print(len(df))


df = df[(df.pgc1 != df.PGC1)]

df[["pgc1", "PGC1"]]

Index([u'Vh', u'Vls_cf4', u'V3k', u'Vmod', u'f_cosmology', u'logWimx',
       u'logWimx_e', u'inc', u'Sqlt', u'Wqlt', u'DM_best', u'dDM_best', u'DMr',
       u'dDMr', u'RMS_r', u'DMr_raw', u'DMi', u'dDMi', u'RMS_i', u'DMi_raw',
       u'DMz', u'dDMz', u'RMS_z', u'DMz_raw', u'DMw1', u'dDMw1', u'RMS_w1',
       u'DMw1_raw', u'ggg', u'rrr', u'iii', u'zzz', u'w1', u'w2', u'RA',
       u'DEC', u'gl', u'gb', u'sgl', u'sgb', u'pgc1', u'Nest_cf4', u'Vmodg',
       u'Nkt', u'N15', u'sglg', u'sgbg', u'RS', u'P0', u'DM_src', u'Dist',
       u'PGC1', u'Nest_cf2fp', u'Name', u'RAJ', u'DeJ', u'Glon', u'Glat',
       u'SGL', u'SGB', u'Ty', u'Asfd', u'Asf', u'Btot', u'Itot', u'Ks', u'b/a',
       u'Vhel', u'Vgsr', u'Vls_cf2fp', u'Vcmb', u'Icnt', u'2M++', u'Fil', u'N',
       u'DMav', u'Hav', u'logH', u'Dsmc', u'DMsmc', u'Denr', u'DMenr', u'Dfar',
       u'DMfar', u'logV', u'Vcl', u'NameS', u'NameN', u'NameF', u'Lclust',
       u'Bclust'],
      dtype='object')
3


,pgc1,PGC1
PGC,,


In [32]:
df = pd.read_csv('all_cf2_fp.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_cf2fp = df.set_index('PGC')

df = df_6dfgs.join(df_cf2fp, on='PGC', how='inner', lsuffix='_6dfgs', rsuffix='_cf2fp')

print(df.columns)
print(len(df))


df = df[(df.PGC1_6dfgs != df.PGC1_cf2fp)]

df[["PGC1_6dfgs", "PGC1_cf2fp"]]

Index([u'PGC1_6dfgs', u'Nest_6dfgs', u'Name_6dfgs', u'Vcmb_6dfgs', u'Vgp',
       u'D75', u'D100', u'feD', u'DzDh', u'eDz', u'skew', u'RA', u'DE', u'Vg',
       u'Gp', u'DM_6dfgs', u'eDM_6dfgs', u'Dist', u'PGC1_cf2fp', u'Nest_cf2fp',
       u'Name_cf2fp', u'RAJ', u'DeJ', u'Glon', u'Glat', u'SGL', u'SGB', u'Ty',
       u'Asfd', u'Asf', u'Btot', u'Itot', u'Ks', u'b/a', u'Vhel', u'Vgsr',
       u'Vls', u'Vcmb_cf2fp', u'Icnt', u'2M++', u'Fil', u'N', u'DMav', u'Hav',
       u'logH', u'Dsmc', u'DMsmc', u'Denr', u'DMenr', u'Dfar', u'DMfar',
       u'logV', u'Vcl', u'NameS', u'NameN', u'NameF', u'Lclust', u'Bclust'],
      dtype='object')
144


,PGC1_6dfgs,PGC1_cf2fp
PGC,,


In [33]:
df = pd.read_csv('CF4TFRcomponents.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_ALLcf4tfr = df.set_index('PGC')

In [34]:
df = pd.read_csv('ALLcf3SNIa.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_tmp = df.set_index('PGC')

df = df_ALLcf4tfr.join(df_tmp, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_ALLcf3SNIa')

print(df.columns)
print(len(df))


df = df[(df.PGC1_cf4 != df.PGC1_ALLcf3SNIa)]

df[["PGC1_cf4", "PGC1_ALLcf3SNIa"]]

Index([u'PGC1_cf4', u'Nest_cf4', u'Vcmb_cf4', u'Vmod', u'DM_av', u'eDM_av',
       u'av_flag', u'DM_cf4', u'eDM_cf4', u'cf4_flag', u'DM_spitzer',
       u'eDM_spitzer', u'spitzer_flag', u'DM_cf2', u'eDM_cf2', u'cf2_flag',
       u'DM_sfi', u'eDM_sfi', u'sfi_flag', u'DM_2mtf', u'eDM_2mtf',
       u'flat_2mtf', u'DM_flat', u'eDM_flat', u'flat_flag', u'PGC1_ALLcf3SNIa',
       u'Nest_ALLcf3SNIa', u'Dist', u'Name', u'RAJ', u'DeJ', u'Glon', u'Glat',
       u'SGL', u'SGB', u'Ty', u'Asfd', u'Asf', u'Btot', u'Itot', u'Ks', u'b/a',
       u'Vhel', u'Vgsr', u'Vls', u'Vcmb_ALLcf3SNIa', u'Icnt', u'2M++', u'Fil',
       u'SNIa', u'N', u'DMav', u'DMr', u'eDMr', u'DMw', u'DMu', u'eDMu',
       u'DMj', u'eDMj', u'DMp', u'eDMp', u'DMh', u'DMf', u'NBG_ID'],
      dtype='object')
87


,PGC1_cf4,PGC1_ALLcf3SNIa
PGC,,
18747,18747,0
20513,20513,0
39920,39920,0
43011,43011,0
47514,47514,0
68364,68364,0
69095,69095,69148


In [35]:
df = pd.read_csv('ScolnicSNIa3Tables.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_tmp = df.set_index('PGC')

df = df_ALLcf4tfr.join(df_tmp, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_ScolnicSNIa3')

print(df.columns)
print(len(df))


df = df[(df.PGC1_cf4 != df.PGC1_ScolnicSNIa3)]

df[["PGC1_cf4", "PGC1_ScolnicSNIa3"]]

Index([u'PGC1_cf4', u'Nest_cf4', u'Vcmb', u'Vmod', u'DM_av', u'eDM_av',
       u'av_flag', u'DM_cf4', u'eDM_cf4', u'cf4_flag', u'DM_spitzer',
       u'eDM_spitzer', u'spitzer_flag', u'DM_cf2', u'eDM_cf2', u'cf2_flag',
       u'DM_sfi', u'eDM_sfi', u'sfi_flag', u'DM_2mtf', u'eDM_2mtf',
       u'flat_2mtf', u'DM_flat', u'eDM_flat', u'flat_flag', u'SNIa',
       u'PGC1_ScolnicSNIa3', u'Nest_ScolnicSNIa3', u'Tab', u'DM', u'DMp',
       u'eDMp', u'DM18', u'eDM18', u'DM17', u'eDM17', u'Vhel', u'Vsn', u'Glon',
       u'Glat', u'SGL', u'SGB'],
      dtype='object')
59


,PGC1_cf4,PGC1_ScolnicSNIa3
PGC,,
69095,69095,69148


In [36]:
df = pd.read_csv('CSP1Burns18.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_tmp = df.set_index('PGC')

df = df_ALLcf4tfr.join(df_tmp, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_CSP1Burns18')

print(df.columns)
print(len(df))


df = df[(df.PGC1_cf4 != df.PGC1_CSP1Burns18)]

df[["PGC1_cf4", "PGC1_CSP1Burns18"]]

Index([u'PGC1_cf4', u'Nest_cf4', u'Vcmb', u'Vmod', u'DM_av', u'eDM_av',
       u'av_flag', u'DM_cf4', u'eDM_cf4', u'cf4_flag', u'DM_spitzer',
       u'eDM_spitzer', u'spitzer_flag', u'DM_cf2', u'eDM_cf2', u'cf2_flag',
       u'DM_sfi', u'eDM_sfi', u'sfi_flag', u'DM_2mtf', u'eDM_2mtf',
       u'flat_2mtf', u'DM_flat', u'eDM_flat', u'flat_flag', u'SNIa',
       u'PGC1_CSP1Burns18', u'Nest_CSP1Burns18', u'muCV', u'emu', u'zh',
       u'z3k', u'sBV', u'esBV', u'Dm15', u'eDm15', u'Vmax', u'eVmax', u'EBV',
       u'eEBV', u'Rv', u'eRv', u'covER', u'Q', u'RA', u'Dec', u'glon', u'glat',
       u'sgl', u'sgb'],
      dtype='object')
30


,PGC1_cf4,PGC1_CSP1Burns18
PGC,,


In [37]:
df = pd.read_csv('lossGanesh13.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_tmp = df.set_index('PGC')

df = df_ALLcf4tfr.join(df_tmp, on='PGC', how='inner', lsuffix='_cf4', rsuffix='_lossGanesh13')

print(df.columns)
print(len(df))


df = df[(df.PGC1_cf4 != df.PGC1_lossGanesh13)]

df[["PGC1_cf4", "PGC1_lossGanesh13"]]

Index([u'PGC1_cf4', u'Nest_cf4', u'Vcmb', u'Vmod', u'DM_av', u'eDM_av',
       u'av_flag', u'DM_cf4', u'eDM_cf4', u'cf4_flag', u'DM_spitzer',
       u'eDM_spitzer', u'spitzer_flag', u'DM_cf2', u'eDM_cf2', u'cf2_flag',
       u'DM_sfi', u'eDM_sfi', u'sfi_flag', u'DM_2mtf', u'eDM_2mtf',
       u'flat_2mtf', u'DM_flat', u'eDM_flat', u'flat_flag', u'SNIa',
       u'PGC1_lossGanesh13', u'Nest_lossGanesh13', u'RA', u'Dec', u'z3k', u'B',
       u'eB', u'x1', u'ex1', u'c', u'ec', u'mu', u'emu', u'Sample', u'Ref'],
      dtype='object')
48


,PGC1_cf4,PGC1_lossGanesh13
PGC,,
69095,69095,69148


In [38]:
df = pd.read_csv('ALLcf3SNIa.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())

df['pgc'] = df['PGC']
df_ALLcf3SNIa = df.set_index('PGC')

df_ALLcf3SNIa["PGC1"] = df_ALLcf3SNIa.apply(lambda x: x.PGC1 if x.PGC1 != 0 else x.pgc, axis=1)


df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"Vcmb": "Vcmb_ALLcf3SNIa"})
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"PGC1": "PGC1_ALLcf3SNIa"})
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMav": "DM_ALLcf3SNIa"})
df_ALLcf3SNIa["eDM_ALLcf3SNIa"] = 0.2
df_ALLcf3SNIa["DM_ALLcf3SNIa"][df_ALLcf3SNIa["DM_ALLcf3SNIa"]==0] = np.nan
df_ALLcf3SNIa["eDM_ALLcf3SNIa"][df_ALLcf3SNIa["DM_ALLcf3SNIa"].isna()] = np.nan

#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_r"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_r"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMr": "DM_cf3SNIa_r"})
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"eDMr": "eDM_cf3SNIa_r"})
df_ALLcf3SNIa["DM_cf3SNIa_r"][df_ALLcf3SNIa["DM_cf3SNIa_r"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_r"][df_ALLcf3SNIa["DM_cf3SNIa_r"].isna()] = np.nan
#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_w"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_w"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMw": "DM_cf3SNIa_w"})
df_ALLcf3SNIa["eDM_cf3SNIa_w"] = 0.2
df_ALLcf3SNIa["DM_cf3SNIa_w"][df_ALLcf3SNIa["DM_cf3SNIa_w"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_w"][df_ALLcf3SNIa["DM_cf3SNIa_w"].isna()] = np.nan
#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_u"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_u"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMu": "DM_cf3SNIa_u"})
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"eDMu": "eDM_cf3SNIa_u"})
df_ALLcf3SNIa["DM_cf3SNIa_u"][df_ALLcf3SNIa["DM_cf3SNIa_u"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_u"][df_ALLcf3SNIa["DM_cf3SNIa_u"].isna()] = np.nan
#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_j"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_j"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMj": "DM_cf3SNIa_j"})
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"eDMj": "eDM_cf3SNIa_j"})
df_ALLcf3SNIa["DM_cf3SNIa_j"][df_ALLcf3SNIa["DM_cf3SNIa_j"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_j"][df_ALLcf3SNIa["DM_cf3SNIa_j"].isna()] = np.nan
#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_p"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_p"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMp": "DM_cf3SNIa_p"})
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"eDMp": "eDM_cf3SNIa_p"})
df_ALLcf3SNIa["DM_cf3SNIa_p"][df_ALLcf3SNIa["DM_cf3SNIa_p"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_p"][df_ALLcf3SNIa["DM_cf3SNIa_p"].isna()] = np.nan
#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_h"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_h"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMh": "DM_cf3SNIa_h"})
df_ALLcf3SNIa["eDM_cf3SNIa_h"] = 0.2
df_ALLcf3SNIa["DM_cf3SNIa_h"][df_ALLcf3SNIa["DM_cf3SNIa_h"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_h"][df_ALLcf3SNIa["DM_cf3SNIa_h"].isna()] = np.nan
#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_f"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_f"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMf": "DM_cf3SNIa_f"})
df_ALLcf3SNIa["eDM_cf3SNIa_f"] = 0.2
df_ALLcf3SNIa["DM_cf3SNIa_f"][df_ALLcf3SNIa["DM_cf3SNIa_f"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_f"][df_ALLcf3SNIa["DM_cf3SNIa_f"].isna()] = np.nan

# "ALLcf3SNIa", "cf3SNIa_w"
catalogs = ["cf3SNIa_r", "cf3SNIa_u", "cf3SNIa_j","cf3SNIa_p", "cf3SNIa_h","cf3SNIa_f", "cf3SNIa_w"]

ss = []
for cat in catalogs:
    ss += ['PGC1_'+cat, "DM_"+cat, "eDM_"+cat, "Vcmb_"+cat]

df = df_ALLcf3SNIa[ss]

print(len(df))

df.head()

/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: SettingWit

389


,PGC1_cf3SNIa_r,DM_cf3SNIa_r,eDM_cf3SNIa_r,Vcmb_cf3SNIa_r,PGC1_cf3SNIa_u,DM_cf3SNIa_u,eDM_cf3SNIa_u,Vcmb_cf3SNIa_u,PGC1_cf3SNIa_j,DM_cf3SNIa_j,...,eDM_cf3SNIa_h,Vcmb_cf3SNIa_h,PGC1_cf3SNIa_f,DM_cf3SNIa_f,eDM_cf3SNIa_f,Vcmb_cf3SNIa_f,PGC1_cf3SNIa_w,DM_cf3SNIa_w,eDM_cf3SNIa_w,Vcmb_cf3SNIa_w
PGC,,,,,,,,,,,,,,,,,,,,,
250,250,34.93,0.16,6905,250,34.34,0.18,6905,250,NaN,...,0.2,6905,250,NaN,NaN,6905,250,NaN,NaN,6905
282,282,35.95,0.21,11001,282,NaN,NaN,11001,282,NaN,...,NaN,11001,282,NaN,NaN,11001,282,NaN,NaN,11001
415,415,34.97,0.17,7183,415,34.44,0.19,7183,415,NaN,...,0.2,7183,415,NaN,NaN,7183,415,NaN,NaN,7183
515,515,NaN,NaN,4897,515,33.70,0.21,4897,515,NaN,...,0.2,4897,515,NaN,NaN,4897,515,NaN,NaN,4897
963,963,NaN,NaN,3607,963,32.64,0.28,3607,963,32.87,...,NaN,3607,963,NaN,NaN,3607,963,NaN,NaN,3607


In [39]:
catalogs = ["cf3SNIa_p"] #, "cf3SNIa_j"] #,"cf3SNIa_p", "cf3SNIa_h","cf3SNIa_f"]

ss = []
for cat in catalogs:
    ss += ["DM_"+cat, "eDM_"+cat]

df_ = df[ss].dropna()

# 'CSP1Burns18.csv'
# 'lossGanesh13.csv'
# 'ScolnicSNIa3Tables.csv'
df_tmp = pd.read_csv('ScolnicSNIa3Tables.csv', delimiter='|')
df_tmp = df_tmp.rename(columns=lambda x: x.strip())
df_tmp = df_tmp.set_index('PGC')

dfj = df_tmp.join(df_, how='inner', lsuffix='_r', rsuffix='_l')

dfj

,SNIa,PGC1,Nest,Tab,DM,DMp,eDMp,DM18,eDM18,DM17,eDM17,Vhel,Vsn,Glon,Glat,SGL,SGB,DM_cf3SNIa_p,eDM_cf3SNIa_p
PGC,,,,,,,,,,,,,,,,,,,
963,1998dk,963,211877,1,33.339,33.339,0.191,33.350,0.168,33.366,0.114,3961,3625,102.8519,-62.1619,292.9641,9.5361,32.74,0.05
1583,2007A,1583,210125,1,34.537,34.537,0.151,34.523,0.122,34.539,0.080,5185,4884,113.0916,-49.4875,307.0273,10.4740,33.15,0.06
3508,1999ef,3513,200676,1,36.357,36.357,0.120,36.348,0.087,36.342,0.075,11660,11473,125.7106,-50.0840,308.9798,2.5480,35.70,0.04
3773,1998ef,4224,200005,1,33.991,33.991,0.147,34.013,0.129,34.012,0.092,5302,4983,125.8793,-30.5657,328.1438,6.5752,33.27,0.11
3981,2000dk,4224,200005,3,34.244,0.000,0.000,0.000,0.000,34.244,0.091,5223,4803,126.8353,-30.3470,328.5190,5.8085,33.53,0.01
5037,1999ej,5098,200006,1,34.309,34.309,0.164,34.305,0.145,34.305,0.106,4104,4593,130.4359,-28.9420,330.4440,2.9656,33.48,0.04
5577,1993ae,5324,200011,1,34.416,34.416,0.147,34.440,0.128,34.437,0.094,5897,5252,144.6250,-63.2283,296.8903,-8.9063,33.59,0.03
6007,1998es,6007,211623,2,32.941,0.000,0.000,32.941,0.205,0.000,0.000,3165,3178,143.1891,-55.1804,305.0555,-8.5059,32.17,0.02
9618,1999gp,9618,207684,3,35.306,0.000,0.000,0.000,0.000,35.306,0.069,8018,7807,143.2545,-19.5054,341.0890,-7.6319,34.57,0.03


In [57]:
df1 = pd.read_csv('Tonry_sbf.csv', delimiter='|')
df1 = df1.rename(columns=lambda x: x.strip())
df1 = df1.set_index('PGC')

df2 = pd.read_csv('VirgoFornax_sbf.csv', delimiter='|')
df2 = df2.rename(columns=lambda x: x.strip())
df2 = df2.set_index('PGC')

df3 = pd.read_csv('Hydra_Centaurus_sbf.csv', delimiter='|')
df3 = df3.rename(columns=lambda x: x.strip())
df3 = df3.set_index('PGC')

df4 = pd.read_csv('Cantiello18_sbf.csv', delimiter='|')
df4 = df4.rename(columns=lambda x: x.strip())
df4 = df4.set_index('PGC')

df5 = pd.read_csv('HST_IR_sbf.csv', delimiter='|')
df5 = df5.rename(columns=lambda x: x.strip())
df5 = df5.set_index('PGC')




In [58]:
df = df1.join(df4, on='PGC', how='inner', lsuffix='_l', rsuffix='_r')

print(df.columns)
print(len(df))


df = df[(df.PGC1_l != df.PGC1_r)]

df[["PGC1_l", "PGC1_r"]]

Index([u'objname', u'Modc', u'PGC1_l', u'Nest_l', u'al2000', u'de2000',
       u'V_hel', u'T', u'GrpID', u'AgB', u'VmI', u'e_VmI', u'N_VmI', u'Ibar',
       u'e_Ibar', u'N_Ibar', u'Mod01', u'e_Mod', u'Rav', u'ratio_in',
       u'ratio_out', u'Qmax', u'PD', u'NbarI', u'Sigma', u'Qual', u'icnt',
       u'NBG_ID', u'LH_ID', u'fil', u'Name', u'VCC', u'dist', u'eD', u'DM',
       u'edm', u'Vhel', u'PGC1_r', u'Nest_r', u'Gp', u'Ty', u'RA_DEJ', u'glon',
       u'glat', u'sgl', u'sgb', u'Ag', u'Area', u'Rad', u'uz', u'euz', u'gi',
       u'egi', u'gz', u'egz', u'Mbar', u'eMbar'],
      dtype='object')
33


,PGC1_l,PGC1_r
PGC,,
